In [ ]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

In [ ]:
d0 = 20
dpy = 2 # CD: 10 ; PIG: 20



In [ ]:
#ds0 = xr.open_dataset(f'../results/CD_1.0_tanh_Tdeep0.5_ztcl-500_{d0:03.0f}.nc')
ds0 = xr.open_dataset(f'../results/PIG_1.0_tanh_Tdeep1.0_ztcl-500_{d0:03.0f}.nc')


for d,d1 in enumerate(np.arange(20,112)):

    #ds = xr.open_dataset(f'../results/CD_1.0_tanh_Tdeep0.5_ztcl-500_{d1:03.0f}.nc')
    ds = xr.open_dataset(f'../results/PIG_1.0_tanh_Tdeep1.0_ztcl-500_{d1:03.0f}.nc')

    Nyears = (d1-d0)/dpy

    fig,ax = plt.subplots(1,2,figsize=(8,7))

    for n in [0,1]:
        ax[n].pcolormesh(ds0.mask,cmap='cmo.ice',vmin=-3,vmax=3)
            
    im = ax[0].pcolormesh(np.where(ds.tmask==1,ds.zb,np.nan),vmin=-1500,vmax=0,cmap='gist_stern_r')
    cb = plt.colorbar(im,ax=ax[0],orientation='horizontal')
    cb.set_label('Draft [m]')

    im = ax[1].pcolormesh(np.where(ds.tmask==1,ds.melt,np.nan),norm=mpl.colors.LogNorm(vmin=1, vmax=100),cmap='inferno')
    cb = plt.colorbar(im,ax=ax[1],orientation='horizontal')
    cb.set_label('Melt [m/yr]')
    ax[1].set_title(f'Mean melt: {np.nanmean(np.where(ds.tmask==1,ds.melt,np.nan)):+2.1f} m/yr')


    ax[0].set_title(f'Year {Nyears: 2.2f}')


    for m in [0,1]:
        ax[m].set_xticks([])
        ax[m].set_yticks([])
        ax[m].set_aspect('equal','box')

    ds.close()

    #plt.savefig(f'../figures/CD/frame_{d:03.0f}')
    plt.savefig(f'../figures/PIG/frame_{d:03.0f}')
    print(d,Nyears)
    plt.close()

ffmpeg -i frame_%03d.png -pix_fmt yuv420p movie_Ah25.mp4